In [31]:
import duckdb
import os

In [32]:
file_path = 'data_cc/cc_tls_features.parquet'

if os.path.exists(file_path):
    print(f"File found: {file_path}")
else:
    print(f"Error: File not found: {file_path}")

File found: data_cc/cc_tls_features.parquet


In [33]:
con = duckdb.connect(":memory:")

In [34]:
query1 = f"""select count(1) from '{file_path}'"""
result1 = con.execute(query1).fetchdf()


In [35]:
query2 = f"""
SELECT count(1) from
    (SELECT visit_id, domain_name FROM '{file_path}' group by visit_id, domain_name)
"""
result2 = con.execute(query2).fetchdf()


In [36]:
query3 = f"""select count(1) from '{file_path}' where connect_ok is True"""
result3 = con.execute(query3).fetchdf()


In [37]:
query4 = f"""
SELECT count(1) from
    (SELECT visit_id, domain_name FROM '{file_path}' where connect_ok is True group by visit_id, domain_name)
"""
result4 = con.execute(query4).fetchdf()


In [38]:
query5 = f"""
SELECT count(1) from
    (SELECT visit_id, domain_name
    FROM '{file_path}'
    where connect_ok
    and highest_version_supported is not Null
    and lowest_version_supported is not NUll
    is True group by visit_id, domain_name)
"""
result5 = con.execute(query5).fetchdf()


In [39]:
query6 = f"""
SELECT high, low, count(1) from
    (SELECT visit_id, domain_name, max(highest_version_supported) as high, max(lowest_version_supported) as low
    FROM '{file_path}'
    where connect_ok
    and highest_version_supported is not Null
    and lowest_version_supported is not NUll
    is True group by visit_id, domain_name) group by high, low
"""
result6 = con.execute(query6).fetchdf()


In [40]:
print("Query 1: Total count of records")
display(result1)

print("Query 2: Count of unique visit_id and domain_name combinations")
display(result2)

print("Query 3: Number or records with 'connect_ok' is True'")
display(result3)

print("Query 4: Count of unique visit_id and domain_name combinations with successful connections")
display(result4)

print("Query 5: Count of unique visit_id and domain_name combinations with valid version information (low / high column not Null)")
display(result5)

print("Query 6: Distribution of highest and lowest supported TLS versions")
display(result6)



Query 1: Total count of records


,count(1)
0,5280


Query 2: Count of unique visit_id and domain_name combinations


,count(1)
0,2640


Query 3: Number or records with 'connect_ok' is True'


...

,count(1)
0,1006


Query 5: Count of unique visit_id and domain_name combinations with valid version information (low / high column not Null)


,count(1)
0,959


Query 6 Distribution of highest and lowest supported TLS versions


,high,low,count(1)
0,TLSv1.2,TLSv1.2,44
1,TLSv1.3,TLSv1.3,15
2,TLSv1.2,TLSv1,93
3,TLSv1.2,TLSv1.1,2
4,TLSv1,SSLv2,1
5,TLSv1.3,TLSv1.1,133
6,TLSv1.3,TLSv1,425
7,TLSv1.2,SSLv3,2
8,TLSv1.1,TLSv1,1
9,TLSv1.3,TLSv1.2,243
